In [1]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Function
import torchvision
import time
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as util_data
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models
import cv2
import time

import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from typing import Any
import random
import torchvision

In [2]:
dataset_root = '/kaggle/input/poly-dataa/poly_ll'
#dataset_root = ‘../input/datagood’

dataset_train_root = os.path.join(dataset_root, 'train')
dataset_val_root = os.path.join(dataset_root, 'test/')
dataset_all_dataset_root = os.path.join(dataset_root, 'all_dataset')


# win设为0 
num_workers = 8

num_threads =8 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
num_epochs =100
# epoch_lr_decrease = 100
learning_rate = 0.004
encode_length =256
num_classes =386

In [4]:
# 配置训练集DataSet类
class SessionDataset(Dataset):
    
    # root数据集根目录
    def __init__(self, root, transform=None):
        # 对继承自父类的属性进行初始化
#         super(Oxford102Dataset, self).__init__()
        
        #读取图像标签  [  0   0   0 ... 599 599 599]
        self.labels = np.array([int(x.split('_')[0]) for x in os.listdir(path=root)])

        # 构造图像路径

        self.image_files = np.array([x.path for x in os.scandir(path=root)])
        
        # 数据增强
        self.transform = transform
        
    
    def __getitem__(self, index):
        
        # 根据Index读取图像
        img = cv2.imread(self.image_files[index])
        
        #根据index读取标签
        label = self.labels[index]


        if self.transform:
            img = self.transform(img)
        
        return img, label
    
    
    def __len__(self):
        return np.size(self.image_files)
        

In [5]:
data_transform = {
    "train": transforms.Compose([
        transforms.ToPILImage(),#将数据类型转变为图片数据
#         transforms.RandomResizedCrop((224, 224)),#随机裁剪224*224
        transforms.RandomResizedCrop((128, 128)),#随机裁剪224*224
        transforms.RandomHorizontalFlip(),#水平防线随机反转
        transforms.ToTensor(),#转为Tensor
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626)),#标准化处理
        
    ]),
    "val": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    "all_dataset": transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize((0.51568358, 0.51568358, 0.51568358), (0.10332626, 0.10332626, 0.10332626))

    ]),
    
}

In [6]:
train_batch_size = 64#64
val_batch_size = 64




session_train_dataset = SessionDataset(root=dataset_train_root,
                                    transform=data_transform['train'])

train_loader = DataLoader(session_train_dataset,
                         batch_size=train_batch_size,
                         shuffle=True,
                         num_workers=num_workers)


session_val_dataset = SessionDataset(root=dataset_val_root,
                                    transform=data_transform['val'])
val_loader = DataLoader(session_val_dataset,
                         batch_size=val_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

# 包含整个验证集

sessiondatabaseDataset = SessionDataset(root=dataset_all_dataset_root,
                                    transform=data_transform['all_dataset'])

database_loader = DataLoader(sessiondatabaseDataset,
                         batch_size=train_batch_size,
                         shuffle=False,
                         num_workers=num_workers)

train_num = len(session_train_dataset)#4800
val_num = len(session_val_dataset)#1200
datasetloader_num = len(sessiondatabaseDataset)#6000
print(train_num, val_num,datasetloader_num)


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


3088 772 3860


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the basic residual block
class BasicBlock(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * self.expansion)
            )

    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = F.relu(out)
        out = self.bn2(self.conv2(out))
        out = F.relu(out)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# Define the ResNet-50 model
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        layers = []
        layers.append(block(self.inplanes, planes, stride))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# Create an instance of the ResNet-50 model
def original_resnet50(pretrained=False, progress=True, **kwargs):
    return ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)

In [8]:
import numpy as np

 


def logistic(x, k=1, x0=0):
    """
    Logistic function: L / (1 + exp(-k * (x - x0)))
    
    Parameters:
    - x: Input values
    - k: Steepness of the curve (default: 1)
    - x0: x-value of the sigmoid's midpoint (default: 0)
    
    Returns:
    - Result of the logistic function
    """
    return 1 / (1 + np.exp(-k * (x - x0)))


def new_dropout(x, level=0.5):  
    if level < 0. or level >= 1:#level是概率值，必须在0~1之间  
        raise Exception('Dropout level must be in interval [0, 1].')  
    retain_prob = 1. - level  
    #我们通过binomial函数，生成与x一样的维数向量。binomial函数就像抛硬币一样，我们可以把每个神经元当做抛硬币一样  
    #硬币 正面的概率为p，n表示每个神经元试验的次数  
    #因为我们每个神经元只需要抛一次就可以了所以n=1，size参数是我们有多少个硬币。 
     
   # sample=nn.random.binomial(n=1,p=retain_prob,size=x.shape)#即将生成一个0、1分布的向量，0表示这个神经元被屏蔽，不工作了，也就是dropout了  
    sample = torch.from_numpy(np.array(logistic(0.3, 1000))).to(device)

#
  #  sample=np.array(logistic(0.36,x.dim()))#
#     print (sample)  
#     print(type(sample))
    
    x *=sample          #屏蔽某些神经元
    
    x /= retain_prob   #此处是dropout
    return x



In [9]:


# new layer
class hash(Function):
    @staticmethod
    def forward(ctx, input):
        #ctx.save_for_backward(input)
        #使用在pytroch中的sign函数得到二值哈希码
        return torch.sign(input)    
    @staticmethod
    def backward(ctx, grad_output):
        #input,  = ctx.saved_tensors
        #grad_output = grad_output.data
        return grad_output
def hash_layer(input):
    return hash.apply(input)


    
    
class CNN_ResNet(nn.Module):
    def __init__(self, encode_length, num_classes):
        super(CNN_ResNet, self).__init__()
        self.res = original_resnet50(pretrained=False,num_classes=1000).to(device)
    
        self.res.fc = nn.Linear(in_features=2048, out_features=1000, bias=True)
      
        self.fc_plus = nn.Linear(1000, encode_length) 
        
        self.fc = nn.Linear(encode_length, num_classes, bias=False)

    def forward(self, x):
        
        x = self.res(x)
        #x = new_dropout(x)
        x = self.fc_plus(x)

        code = hash_layer(x)
   
        output = self.fc(code)
        
  
        return output, x, code
    

#net = CNN_ResNet(encode_length=encode_length, num_classes=num_classes).to(device)
print('ok')

ok


In [10]:
import torch
import torch.nn as nn

class SecureRandomProjectionLayer(nn.Module):
    def __init__(self, input_dim, output_dim, num_classes):
        super(SecureRandomProjectionLayer, self).__init__()
        self.random_projection_matrices = nn.Parameter(torch.randn(num_classes, input_dim, output_dim) * 0.01, requires_grad=False)

    def forward(self, x, class_labels):
        class_specific_matrices = self.random_projection_matrices[class_labels]
        transformed_data = torch.matmul(x, class_specific_matrices)
        return transformed_data

class hash(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return torch.sign(input)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

def hash_layer(input):
    return hash.apply(input)

class CNN_ResNet(nn.Module):
    def __init__(self, encode_length, num_classes):
        super(CNN_ResNet, self).__init__()
        self.res = original_resnet50(pretrained=False, num_classes=1000).to(device)
        self.res.fc = nn.Linear(in_features=2048, out_features=1000, bias=True)
        self.secure_projection = SecureRandomProjectionLayer(1000, 1000,num_classes)
        self.fc_plus = nn.Linear(1000, 1024)
        self.fc = nn.Linear(1024, num_classes, bias=False)

    def forward(self, x, class_labels=None):
        x = self.res(x)
        x = new_dropout(x)
        if class_labels is not None:
            class_specific_matrices = self.secure_projection.random_projection_matrices[class_labels]
            x = torch.matmul(x, class_specific_matrices)
        if x.dim() != 3:
            x = x.view(x.size(0), -1, 1)

        n, j = x.shape[1], x.shape[2]
        x = x.view(x.size(0), n, j)
        max_values, _ = x.max(dim=2)
        x = self.fc_plus(max_values)
       # code = hash_layer(x)
        code=x
        
        # If class_labels are provided, use the class-specific projection matrix
        if class_labels is not None:
            class_specific_matrices = self.secure_projection.random_projection_matrices[class_labels]
            x = torch.matmul(x, class_specific_matrices)
        
        output = self.fc(code)
        
        return output, x, code

# Example usage
 
num_projection_classes =60
net = CNN_ResNet(encode_length=encode_length, num_classes=num_classes).to(device)
print('ok')

ok


In [11]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)




In [12]:
def compress(train, test, model, classes=386):
    retrievalB = list([])
    retrievalL = list([])
    with torch.no_grad():
        for batch_step, (data, target) in enumerate(train):
            
            var_data = data.to(device)
            
            # code为网络输出的哈希码
            _,_, code = model(var_data)
            
            retrievalB.extend(code.cpu().data.numpy())
            retrievalL.extend(target)
            
        queryB=list([])
        queryL=list([])
        for batch_step, (data, target) in enumerate(test):     
            var_data = data.to(device)
            _,_, code = model(var_data)
            
            queryB.extend(code.cpu().data.numpy())
            queryL.extend(target)
            
        retrievalB=np.array(retrievalB)
        retrievalL=np.eye(classes)[np.array(retrievalL)]
            
        queryB=np.array(queryB)
        queryL=np.eye(classes)[np.array(queryL)]   
        
            
        return retrievalB, retrievalL, queryB, queryL


    
    
def calculate_hamming(B1, B2):
    """
    :param B1:  vector [n]
    :param B2:  vector [r*n]
    :return: hamming distance [r]
    """
    q = B2.shape[1] # max inner product value
    distH = 0.5 * (q - np.dot(B1, B2.transpose()))
    return distH


def calculate_map(qB, rB, queryL, retrievalL):
    """
       :param qB: {-1,+1}^{mxq} query bits
       :param rB: {-1,+1}^{nxq} retrieval bits
       :param queryL: {0,1}^{mxl} query label
       :param retrievalL: {0,1}^{nxl} retrieval label
       :return:
    """
    num_query = queryL.shape[0]
    map = 0
    for iter in range(num_query):
        # gnd : check if exists any retrieval items with same label
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        # tsum number of items with same label
        tsum = np.sum(gnd).astype(int)
        if tsum == 0:
            continue
        # sort gnd by hamming dist
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        count = np.linspace(1, tsum, tsum) # [1,2, tsum]
        tindex = np.asarray(np.where(gnd == 1)) + 1.0
        map_ = np.mean(count / (tindex))
        # print(map_)
        map = map + map_
    map = map / num_query
    return map



def calculate_top_map(qB, rB, queryL, retrievalL, topk):
    """
    :param qB: {-1,+1}^{mxq} query bits
    :param rB: {-1,+1}^{nxq} retrieval bits
    :param queryL: {0,1}^{mxl} query label
    :param retrievalL: {0,1}^{nxl} retrieval label
    :param topk:
    :return:
    """
    num_query = queryL.shape[0]
    topkmap = 0
    for iter in range(num_query):
        
        # 当前测试集标签 乘 训练集标签的转置
        # dot是为找到与当前查询二值哈希码与库中的哈希码相似（或者完全相同）的哈希码 >0的结果为True or False
        # .astype(np.float32)将true变为1， false变为0
        # gnd应该是标签的相似向量1为相似，0为不相似
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        hamm = calculate_hamming(qB[iter, :], rB)
        ind = np.argsort(hamm)
        gnd = gnd[ind]

        tgnd = gnd[0:topk]
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        count = np.linspace(1, tsum, tsum)

        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        topkmap_ = np.mean(count / (tindex))
        # print(topkmap_)
        topkmap = topkmap + topkmap_
    topkmap = topkmap / num_query
    return topkmap


# 计算mAP
# rB整个数据集二值哈希码 qB测试集二值哈希码 retrievalL整个标签 queryL测试集标签（标签都是oneHot编码） topk
def myCalcTopMap(rB, qB, retrievalL, queryL, topk):
    
    # 测试集标签的数量
    num_query = queryL.shape[0]
    
    topkmap = 0
    
    # 计算AP
    for iter in tqdm(range(num_query)):
        
        # 当前测试集标签 乘 训练集标签的转置
        # dot是为找到与当前查询二值哈希码与库中的哈希码相似（或者完全相同）的哈希码 >0的结果为True or False
        # .astype(np.float32)将true变为1， false变为0
        # gnd应该是标签的相似向量1为相似，0为不相似
        gnd = (np.dot(queryL[iter, :], retrievalL.transpose()) > 0).astype(np.float32)
        
        # 计算当前测试集二值哈希码与全部数据集二值哈希码之间的汉明距离
        hamm = CalcHammingDist(qB[iter, :], rB)
        
        # 取排序后的下标（升序）
        ind = np.argsort(hamm)
        
        # 取按照汉明距离对gnd（标签相似度）排序
        gnd = gnd[ind]
        
        # 取前topK个gnd
        # tgnd 1,1,0,0,1..
        tgnd = gnd[0:topk]
        
        # 对tgnd求和--tgnd中非1即0
        # 表示topK中检索正确的个数
        tsum = np.sum(tgnd).astype(int)
        if tsum == 0:
            continue
        
        # linspace(start, stop, num)--1,2,3...tsum
        count = np.linspace(1, tsum, tsum)
        
        # asarray-生成ndarray数组，相比array,当输入为ndarray时，不会重新创建，更省内存
        # tgnd [1,1,,0,0,1..]长度为topk
        # np.where,找到数组tgnd == 1的下标
        # tindex含义为tgnd中值为1的下标数组+1
        tindex = np.asarray(np.where(tgnd == 1)) + 1.0
        
        # count [1,2,3,...topk] tindex [1,3,5...]
        # 其含义是AP?
        topkmap_ = np.mean(count / (tindex))
        # AP累加
        topkmap = topkmap + topkmap_
    # mAP
    topkmap = topkmap / num_query
    
    return topkmap

In [ ]:
# 训练过程
import os
import numpy as np
import torch
from torch.utils.data import DataLoader
from scipy.io import savemat
from tqdm import tqdm

# Define the dataset size explicitly
dataset_size = 3860
counter = 1

# Define the output directory for saving .mat files
output_dir = "all_dataset_features"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the model to evaluation mode
net.eval()

# Train the Model
acc_dict = {}
best = 0.0

torch.set_num_threads(num_threads)

# 根据条件判断是否使用多GPU
if torch.cuda.device_count() > 1:
    net = nn.DataParallel(net)

net.to(device)

# Train the Model
for epoch in range(num_epochs):
    
    net.train()
#     adjust_learning_rate(optimizer, epoch)
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs, feature, _ = net(images)

        loss1 = criterion(outputs, labels)
        loss2 = torch.mean(torch.abs(torch.pow(torch.abs(feature) - torch.ones(feature.size()).to(device), 3)))
        loss = loss1 + 0.1 * loss2
        loss.backward()
        optimizer.step()

        print('Epoch [%d/%d], Iter [%d/%d] Loss1: %.4f Loss2: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(sessiondatabaseDataset) // train_batch_size,
                 loss1.item(), loss2.item()))

    # Test the Model
    net.eval()  # Change model to 'eval' mode
    correct = 0
    total = 0
    with torch.no_grad():
        
        for images, labels in val_loader:
            images = images.to(device)
            outputs, _, _ = net(images)
            _, predicted = torch.max(outputs.cpu().data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum()

    print('Test Accuracy of the model: %.2f %%' % (100.0 * correct / total))

    if 1.0 * correct / total > best:
        best = 1.0 * correct / total
        torch.save(net.state_dict(), 'temp44.pkl')
    print('best: %.2f %%' % (best * 100.0))

    # Save intermediate features after 100 epochs
    if epoch == 99:  # Adjust the epoch number as needed
        net.eval()
        counter_before_save = counter  # Save the current counter value
        with torch.no_grad():
            for images, _ in tqdm(database_loader, desc=f"Saving features at epoch {epoch+1}"):
                images = images.to(device)
                _, intermediate_feature, _ = net(images)
                
                for j in range(len(intermediate_feature)):
                    feature_value = intermediate_feature[j].cpu().numpy()

                    # Modify the file name creation to include the counter and group index
                    group_index = (counter - 1) // 10 + 1
                    counter_within_group = (counter - 1) % 10 + 1
                    file_name = os.path.join(output_dir, f"{group_index}_{counter_within_group}.mat")
                    
                    savemat(file_name, {"feature_value": feature_value})
                    print(f"Saved intermediate feature value to {file_name}")

                    counter += 1  # Increment the counter

                    # Check if we've processed all expected images
                    if counter > dataset_size:
                        break

                if counter > dataset_size:
                    break

        print(f"All {counter - counter_before_save} intermediate features saved at epoch {epoch+1}")

print(f"All {dataset_size} dataset intermediate features saved sequentially.")

    
        


 


/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch [1/100], Iter [1/60] Loss1: 6.0657 Loss2: 0.5406
Epoch [1/100], Iter [2/60] Loss1: 6.0177 Loss2: 0.3681
Epoch [1/100], Iter [3/60] Loss1: 6.1268 Loss2: 0.5108
Epoch [1/100], Iter [4/60] Loss1: 6.0067 Loss2: 0.5368
Epoch [1/100], Iter [5/60] Loss1: 6.0447 Loss2: 0.5769
Epoch [1/100], Iter [6/60] Loss1: 6.1333 Loss2: 0.4078
Epoch [1/100], Iter [7/60] Loss1: 6.2030 Loss2: 0.7816
Epoch [1/100], Iter [8/60] Loss1: 6.0506 Loss2: 0.4898
Epoch [1/100], Iter [9/60] Loss1: 6.1469 Loss2: 0.4981
Epoch [1/100], Iter [10/60] Loss1: 6.0919 Loss2: 0.4222
Epoch [1/100], Iter [11/60] Loss1: 6.2328 Loss2: 0.8821
Epoch [1/100], Iter [12/60] Loss1: 6.0494 Loss2: 0.7721
Epoch [1/100], Iter [13/60] Loss1: 6.0083 Loss2: 0.4216
Epoch [1/100], Iter [14/60] Loss1: 5.9994 Loss2: 0.3930
Epoch [1/100], Iter [15/60] Loss1: 6.1926 Loss2: 0.4890
Epoch [1/100], Iter [16/60] Loss1: 6.1058 Loss2: 0.3800
Epoch [1/100], Iter [17/60] Loss1: 6.1477 Loss2: 0.4176
Epoch [1/100], Iter [18/60] Loss1: 6.1861 Loss2: 0.3850
E

In [ ]:
import shutil

# Directory containing the .mat files
source_dir = "/kaggle/working/all_dataset_features"

# Zip archive file name
zip_file = "hash_codesm1.zip"

# Create a zip archive
shutil.make_archive(zip_file.replace(".zip", ""), 'zip', source_dir)

print(f"Created {zip_file}")

# Move the zip file to the desired download location
os.rename(f"{zip_file.replace('.zip', '')}.zip", "/path/to/download/location/hash_codes1.zip")


### 